# Importing libraries

In [1]:
import sqlite3
import pandas as pd

# Task 1 - Connecting to the database

In [2]:
# Connect to the SQLite database file, this will connect to the DB file
conn = sqlite3.connect('/content/infection.db')

In [3]:
# Create a cursor object to execute SQL queries, this is required to fetch results
cursor = conn.cursor()

**Task 1 - Result**

In [4]:
# Execute SQL queries using the cursor, in this task, we're listing the tables 
tables = print(cursor.execute("SELECT name FROM sqlite_master;").fetchall())

[('MRSA',), ('CDI',), ('COVID19',)]


# Task 2 -  Load the Data into Data Frames

Each table is loaded into separate dataframe.





In [6]:
# Execute the query to retrieve tables
cursor.execute("SELECT name FROM sqlite_master;")
tables = cursor.fetchall()

In [7]:
# Iterate over the tables, this will show the columns for each type so that we get an idea of schema
for table in tables:
    table_name = table[0]
    print("Table Name:", table_name)
    
    # Execute the query to retrieve column information
    cursor.execute("PRAGMA table_info({})".format(table_name))
    
    # Fetch all the column information
    columns = cursor.fetchall()
    print("Schema / Number of columns: ",len(columns))
    
    # Print column information
    for column in columns:
        column_index = column[0]
        column_name = column[1]
        data_type = column[2]
        print("{} : {} : {}".format(column_index, column_name, data_type)) 
    
    print()  # Empty line

Table Name: MRSA
Schema / Number of columns:  7
0 : patient_identifier : INTEGER
1 : age : TEXT
2 : unit : TEXT
3 : room : INTEGER
4 : bed : TEXT
5 : result : TEXT
6 : treatment : TEXT

Table Name: CDI
Schema / Number of columns:  7
0 : patient_identifier : INTEGER
1 : age : TEXT
2 : unit : TEXT
3 : room : INTEGER
4 : bed : TEXT
5 : result : TEXT
6 : treatment : TEXT

Table Name: COVID19
Schema / Number of columns:  7
0 : patient_identifier : INTEGER
1 : age : TEXT
2 : unit : TEXT
3 : room : INTEGER
4 : bed : TEXT
5 : result : TEXT
6 : treatment : TEXT



**Task 2 -Result**

In [8]:
# Execute the query and fetch the data into a DataFrame
df_COVID = pd.read_sql_query("SELECT * FROM COVID19", conn)

In [9]:
# Execute the query and fetch the data into a DataFrame
df_CDI = pd.read_sql_query("SELECT * FROM CDI", conn)

In [10]:
# Execute the query and fetch the data into a DataFrame
df_MRSA = pd.read_sql_query("SELECT * FROM MRSA", conn)

# Task 3 - Analyze the Data
Note - The visulation is done in Tableau. Here it's more of exploratpry data anlysis, tyding things up before exporting the data.

*   Added spearte columns for floor level and facility name
*   Added a new column with infection type
*   Checked for Null Values





In [11]:
# Check the data types of columns
column_data_types = df_MRSA.dtypes

# Print the data types
print(column_data_types)

patient_identifier     int64
age                   object
unit                  object
room                   int64
bed                   object
result                object
treatment             object
dtype: object


The 'unit' column is in the format: “floor level - facility name”. Below, I'm creating two additional columns 'FloorLevel' and 'FacilityName' to sperate the string.

In [12]:
# Split the column into two separate columns
df_COVID[['FloorLevel', 'FacilityName']] = df_COVID['unit'].str.split(' - ', expand=True)
df_CDI[['FloorLevel', 'FacilityName']] = df_CDI['unit'].str.split(' - ', expand=True)
df_MRSA[['FloorLevel', 'FacilityName']] = df_MRSA['unit'].str.split(' - ', expand=True)

Below, I'm adding type of infection to corresponding dataframe, this is going to help later in Tableau. 

In [13]:
# Add a new column and set all values to 'Bo'
df_COVID['InfectionType'] = 'COVID19'
df_CDI['InfectionType'] = 'CDI'
df_MRSA['InfectionType'] = 'MRSA'

Below, I was checking for null values for each table - just to see if there is need to discard / modify any data.

In [ ]:
# Table CDI

# Count null values in each column
null_counts = df_CDI.isnull().sum()

# Print the null value counts
print("Null Value Counts for Table CDI:")
print(null_counts)

In [ ]:
# Table COVID

# Count null values in each column
null_counts = df_COVID.isnull().sum()

# Print the null value counts
print("Null Value Counts for Table COVID:")
print(null_counts)

In [ ]:
# Table MRSA

# Count null values in each column
null_counts = df_MRSA.isnull().sum()

# Print the null value counts
print("Null Value Counts for Table MRSA:")
print(null_counts)

Below getting shape for each tables to know number of rows

In [17]:
# Get the length of the column using shape
column_length = df_MRSA['patient_identifier'].shape[0]
print("No. of rows in MRSA ", column_length)

# Get the length of the column using shape
column_length = df_CDI['patient_identifier'].shape[0]
print("No. of rows in CDI ",column_length)

# Get the length of the column using shape
column_length = df_COVID['patient_identifier'].shape[0]
print("No. of rows in MRSA ",column_length)

No. of rows in MRSA  5000
No. of rows in CDI  8000
No. of rows in MRSA  10000


# Task 4: Develop Insights

**Having worked in software industry and from personal experince, I go by the rule that unless it gives you a competive edge, do not re-invvent the wheel!** 

Tools such as MS Excel, Tableau, PowerBI, etc. are build to do the heavy lifting in making interactive and beautiful visualtions, so save matplotlib for quick exploratory analysis or when absoultely necessary. 

Please refer to the Tableau Dashborad 

In [81]:
# Combine the DataFrames into a single DataFrame
combined_df = pd.concat([df_COVID, df_MRSA, df_CDI])

# Export the combined DataFrame to an Excel file
combined_df.to_excel('combined_dataframes.xlsx', index=False)

# Miscellaneous

Find common patients among all three tables

In [21]:
# Find the common values
common_values = set(df_MRSA['patient_identifier']).intersection(df_COVID['patient_identifier'])

# Convert the set of common values to a list
common_values_list = list(common_values)

# Print the common values
print(len(common_values_list))

38


In [23]:
# Find the common values
common_values_MRSA_COVID = set(df_MRSA['patient_identifier']).intersection(df_COVID['patient_identifier'])
common_values_MRSA_COVID_CDI = set(df_CDI['patient_identifier']).intersection(common_values_MRSA_COVID)

# Convert the set of common values to a list
common_values_MRSA_COVID_CDI = list(common_values_MRSA_COVID_CDI)

# Print the common values
print(len(common_values_MRSA_COVID_CDI))

0


In [78]:
common_values = df_COVID['FacilityName'].isin(df_CDI['FacilityName'])

# Check if any common values exist
if common_values.any():
    print("The columns have some values in common.")
else:
    print("The columns do not have any common values.")

The columns have some values in common.
